## qml.probs

In [15]:
import pennylane as qml
from pennylane import numpy as np


dev1 = qml.device('default.qubit', wires=3)
dev2 = qml.device('default.qubit', wires=3)
dev_swap = qml.device('default.qubit', wires=7)

# qml.density_matrix(wires=[0, 1, 2])
# qml.sample(qml.PauliZ(0))
# qml.expval(qml.PauliZ(0))

@qml.qnode(dev1)
def circuit1():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.Hadamard(wires=2)
    # return qml.state()
    return qml.probs(wires=[0, 1, 2])

@qml.qnode(dev2)
def circuit2():
    qml.PauliX(wires=0)
    # qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.Hadamard(wires=2)
    # return qml.state()
    return qml.probs(wires=[0, 1, 2])

print("Circuit 1:")
print(qml.draw(circuit1)()) # qml.draw_mpl

print("\nCircuit 2:")
print(qml.draw(circuit2)())

# Get the quantum states from each circuit
state1 = circuit1()
state2 = circuit2()

print("State 1 and State 2:")
print(state1, state2)
# Compute the direct fidelity between the two states
# fidelity_direct = np.abs(np.dot(np.conj(state1), state2)) ** 2 # qml.state()
fidelity_direct = np.sum(np.sqrt(state1 * state2)) ** 2 # qml.probs


# SWAP test circuit
@qml.qnode(dev_swap)
def swap_test_circuit():
    qml.Hadamard(wires=0)
    # Prepare the first state on wires 1, 2, 3
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.Hadamard(wires=3)
    
    # Prepare the second state on wires 4, 5, 6
    qml.PauliX(wires=4)
    # qml.Hadamard(wires=4)
    qml.CNOT(wires=[4, 5])
    qml.CNOT(wires=[5, 6])
    qml.Hadamard(wires=6)
    
    # Apply controlled-SWAP operations
    qml.CSWAP(wires=[0, 1, 4])
    qml.CSWAP(wires=[0, 2, 5])
    qml.CSWAP(wires=[0, 3, 6])
    
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)

print("\nSWAP Test Circuit:")
print(qml.draw(swap_test_circuit)())

# Calculate the probability of measuring |0> on the ancillary qubit
probs = swap_test_circuit()
fidelity_swap_test = 2 * probs[0] - 1  # Fidelity estimated from SWAP test

# Print both fidelities
print(f"Fidelity (Direct Calculation): {fidelity_direct}")
print(f"Fidelity (SWAP Test Estimation): {fidelity_swap_test}")

# Assert if both fidelities are approximately equal
assert np.isclose(fidelity_direct, fidelity_swap_test, atol=1e-6), "The fidelities do not match!"
print("The fidelities are approximately equal.")


Circuit 1:
0: ──H─╭●───────┤ ╭Probs
1: ────╰X─╭●────┤ ├Probs
2: ───────╰X──H─┤ ╰Probs

Circuit 2:
0: ──X─╭●───────┤ ╭Probs
1: ────╰X─╭●────┤ ├Probs
2: ───────╰X──H─┤ ╰Probs
State 1 and State 2:
[0.25 0.25 0.   0.   0.   0.   0.25 0.25] [0.  0.  0.  0.  0.  0.  0.5 0.5]

SWAP Test Circuit:
0: ──H──────────╭●────╭●────╭●─────H─┤  Probs
1: ──H─╭●───────├SWAP─│─────│────────┤       
2: ────╰X─╭●────│─────├SWAP─│────────┤       
3: ───────╰X──H─│─────│─────├SWAP────┤       
4: ──X─╭●───────╰SWAP─│─────│────────┤       
5: ────╰X─╭●──────────╰SWAP─│────────┤       
6: ───────╰X──H─────────────╰SWAP────┤       
Fidelity (Direct Calculation): 0.4999999999999996
Fidelity (SWAP Test Estimation): 0.4999999999999989
The fidelities are approximately equal.


## qml.state

In [14]:
import pennylane as qml
from pennylane import numpy as np


dev1 = qml.device('default.qubit', wires=3)
dev2 = qml.device('default.qubit', wires=3)
dev_swap = qml.device('default.qubit', wires=7)

# qml.density_matrix(wires=[0, 1, 2])
# qml.sample(qml.PauliZ(0))
# qml.expval(qml.PauliZ(0))

@qml.qnode(dev1)
def circuit1():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.Hadamard(wires=2)
    return qml.state()

@qml.qnode(dev2)
def circuit2():
    qml.PauliX(wires=0)
    # qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.Hadamard(wires=2)
    return qml.state()

print("Circuit 1:")
print(qml.draw(circuit1)()) # qml.draw_mpl

print("\nCircuit 2:")
print(qml.draw(circuit2)())

# Get the quantum states from each circuit
state1 = circuit1()
state2 = circuit2()

print("State 1 and State 2:")
print(state1, state2)
# Compute the direct fidelity between the two states
fidelity_direct = np.abs(np.dot(np.conj(state1), state2)) ** 2 # qml.state()
# fidelity_direct = np.sum(np.sqrt(state1 * state2)) ** 2 # qml.probs


# SWAP test circuit
@qml.qnode(dev_swap)
def swap_test_circuit():
    qml.Hadamard(wires=0)
    # Prepare the first state on wires 1, 2, 3
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.Hadamard(wires=3)
    
    # Prepare the second state on wires 4, 5, 6
    qml.PauliX(wires=4)
    # qml.Hadamard(wires=4)
    qml.CNOT(wires=[4, 5])
    qml.CNOT(wires=[5, 6])
    qml.Hadamard(wires=6)
    
    # Apply controlled-SWAP operations
    qml.CSWAP(wires=[0, 1, 4])
    qml.CSWAP(wires=[0, 2, 5])
    qml.CSWAP(wires=[0, 3, 6])
    
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)

print("\nSWAP Test Circuit:")
print(qml.draw(swap_test_circuit)())

# Calculate the probability of measuring |0> on the ancillary qubit
probs = swap_test_circuit()
fidelity_swap_test = 2 * probs[0] - 1  # Fidelity estimated from SWAP test

# Print both fidelities
print(f"Fidelity (Direct Calculation): {fidelity_direct}")
print(f"Fidelity (SWAP Test Estimation): {fidelity_swap_test}")

# Assert if both fidelities are approximately equal
assert np.isclose(fidelity_direct, fidelity_swap_test, atol=1e-6), "The fidelities do not match!"
print("The fidelities are approximately equal.")


Circuit 1:
0: ──H─╭●───────┤  State
1: ────╰X─╭●────┤  State
2: ───────╰X──H─┤  State

Circuit 2:
0: ──X─╭●───────┤  State
1: ────╰X─╭●────┤  State
2: ───────╰X──H─┤  State
State 1 and State 2:
[ 0.5+0.j  0.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0.5+0.j -0.5+0.j] [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.70710678+0.j -0.70710678+0.j]

SWAP Test Circuit:
0: ──H──────────╭●────╭●────╭●─────H─┤  Probs
1: ──H─╭●───────├SWAP─│─────│────────┤       
2: ────╰X─╭●────│─────├SWAP─│────────┤       
3: ───────╰X──H─│─────│─────├SWAP────┤       
4: ──X─╭●───────╰SWAP─│─────│────────┤       
5: ────╰X─╭●──────────╰SWAP─│────────┤       
6: ───────╰X──H─────────────╰SWAP────┤       
Fidelity (Direct Calculation): 0.4999999999999998
Fidelity (SWAP Test Estimation): 0.4999999999999989
The fidelities are approximately equal.
